In [1]:
from snowflake.vector import Vector
from snowflake.nodes import *
from snowflake.stencil_compiler import CCompiler, PythonCompiler
import numpy as np
from snowflake.analytics import *
ccompiler = CCompiler()
pycompiler = PythonCompiler()
!ctree -cc

ctree looking for relative cache directories named compiled, checking directories under this one



In [2]:
offset = 2*Vector.index_vector(3)
print(offset)

(2*index_0, 2*index_1, 2*index_2)


In [3]:
swa = SparseWeightArray({offset: 4})

In [4]:
sc = StencilComponent('a', swa)

In [5]:
stencil = Stencil(sc, 'out', ((0, 0, 1),)*3, primary_mesh='out')

In [6]:
ckern = ccompiler.compile(stencil)
pykern = pycompiler.compile(stencil)

In [7]:
in_grid = np.arange(16**3, dtype=np.int).reshape((16, 16, 16))

In [8]:
out_grid = np.empty((8, 8, 8), dtype=np.int)

In [9]:
in_grid

array([[[   0,    1,    2, ...,   13,   14,   15],
        [  16,   17,   18, ...,   29,   30,   31],
        [  32,   33,   34, ...,   45,   46,   47],
        ..., 
        [ 208,  209,  210, ...,  221,  222,  223],
        [ 224,  225,  226, ...,  237,  238,  239],
        [ 240,  241,  242, ...,  253,  254,  255]],

       [[ 256,  257,  258, ...,  269,  270,  271],
        [ 272,  273,  274, ...,  285,  286,  287],
        [ 288,  289,  290, ...,  301,  302,  303],
        ..., 
        [ 464,  465,  466, ...,  477,  478,  479],
        [ 480,  481,  482, ...,  493,  494,  495],
        [ 496,  497,  498, ...,  509,  510,  511]],

       [[ 512,  513,  514, ...,  525,  526,  527],
        [ 528,  529,  530, ...,  541,  542,  543],
        [ 544,  545,  546, ...,  557,  558,  559],
        ..., 
        [ 720,  721,  722, ...,  733,  734,  735],
        [ 736,  737,  738, ...,  749,  750,  751],
        [ 752,  753,  754, ...,  765,  766,  767]],

       ..., 
       [[3328, 3329, 

In [10]:
#out_grid

In [11]:
pykern(out_grid, in_grid)

In [12]:
out_grid[:4, :4, :4]

array([[[   0,    8,   16,   24],
        [ 128,  136,  144,  152],
        [ 256,  264,  272,  280],
        [ 384,  392,  400,  408]],

       [[2048, 2056, 2064, 2072],
        [2176, 2184, 2192, 2200],
        [2304, 2312, 2320, 2328],
        [2432, 2440, 2448, 2456]],

       [[4096, 4104, 4112, 4120],
        [4224, 4232, 4240, 4248],
        [4352, 4360, 4368, 4376],
        [4480, 4488, 4496, 4504]],

       [[6144, 6152, 6160, 6168],
        [6272, 6280, 6288, 6296],
        [6400, 6408, 6416, 6424],
        [6528, 6536, 6544, 6552]]])

In [13]:
in_grid[:8:2, :8:2, :8:2]*4

array([[[   0,    8,   16,   24],
        [ 128,  136,  144,  152],
        [ 256,  264,  272,  280],
        [ 384,  392,  400,  408]],

       [[2048, 2056, 2064, 2072],
        [2176, 2184, 2192, 2200],
        [2304, 2312, 2320, 2328],
        [2432, 2440, 2448, 2456]],

       [[4096, 4104, 4112, 4120],
        [4224, 4232, 4240, 4248],
        [4352, 4360, 4368, 4376],
        [4480, 4488, 4496, 4504]],

       [[6144, 6152, 6160, 6168],
        [6272, 6280, 6288, 6296],
        [6400, 6408, 6416, 6424],
        [6528, 6536, 6544, 6552]]])

In [14]:
get_shadow(stencil)

defaultdict(set, {'a': {(2*index_0, 2*index_1, 2*index_2)}})

In [15]:
validate_stencil(stencil, True)

Output does not coincide with input grids. Analysis Complete.


True

In [16]:
sampling_restriction_with_ghost = Stencil(sc, 'out', ((1, -1, 1),)*3, primary_mesh='out')

In [17]:
csrgkern = ccompiler.compile(sampling_restriction_with_ghost)
pysrgkern = pycompiler.compile(sampling_restriction_with_ghost)

In [18]:
data = np.zeros((34, 34, 34), dtype=np.int)
data[1:-1, 1:-1, 1:-1] = np.arange(32*32*32, dtype=np.int).reshape((32, 32, 32))

In [19]:
output = np.zeros((18, 18, 18), dtype=np.int)
output2 = np.zeros_like(output)

In [20]:
csrgkern(output, data)

In [21]:
output

array([[[     0,      0,      0, ...,      0,      0,      0],
        [     0,      0,      0, ...,      0,      0,      0],
        [     0,      0,      0, ...,      0,      0,      0],
        ..., 
        [     0,      0,      0, ...,      0,      0,      0],
        [     0,      0,      0, ...,      0,      0,      0],
        [     0,      0,      0, ...,      0,      0,      0]],

       [[     0,      0,      0, ...,      0,      0,      0],
        [     0,   4228,   4236, ...,   4340,   4348,      0],
        [     0,   4484,   4492, ...,   4596,   4604,      0],
        ..., 
        [     0,   7812,   7820, ...,   7924,   7932,      0],
        [     0,   8068,   8076, ...,   8180,   8188,      0],
        [     0,      0,      0, ...,      0,      0,      0]],

       [[     0,      0,      0, ...,      0,      0,      0],
        [     0,  12420,  12428, ...,  12532,  12540,      0],
        [     0,  12676,  12684, ...,  12788,  12796,      0],
        ..., 
        [

In [22]:
pysrgkern(output2, data)

In [23]:
output2 == output

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True, 